In [ ]:
import import_ipynb
from demo import *
from pywebio.input import input, FLOAT, input_group
from pywebio.output import put_text, put_processbar, set_processbar, put_file,remove,use_scope,put_collapse,put_image
from pywebio import start_server
import pywebio
import os
from datetime import datetime
import pandas as pd

In [ ]:
def web_work():
    #put_text(pywebio.session.info)
    
    with use_scope('scope1'):
        put_text('Welcome to Novel-Sample cis-element discovery pipeline.\nYou can get familiar with our pipeline and reproduce our experiment with demo setting below：')
        put_text('Reported DNA damage response cis-element')
        put_collapse('Acinetobacter DdaA',
        '''
        Input Taxonomy:
        Acinetobacter

        NCBI Protein ID:
        WP_005017521.1,WP_005405730.1,WP_034675625.1,WP_111280119.1,WP_111280136.1,
        WP_005017085.1,WP_101160495.1,WP_005405985.1,WP_111280201.1,WP_111280215.1,
        WP_111280241.1,WP_111280243.1,WP_111280245.1,WP_111280252.1,WP_111280336.1,
        WP_111280342.1,WP_005016405.1,WP_005016170.1,WP_111280466.1,WP_111280719.1,
        WP_111280773.1,WP_005014868.1,WP_111280906.1,WP_111280918.1,WP_005014692.1,
        WP_005014686.1,WP_005405657.1,WP_005025682.1,WP_005404176.1,WP_005014561.1,
        WP_111281284.1,WP_005014193.1,WP_005020133.1,WP_111281445.1,WP_005019893.1,
        WP_111281515.1,WP_111281570.1,WP_111281616.1,WP_005026719.1,WP_111281678.1,
        WP_005019389.1,WP_111281740.1,WP_005019323.1,WP_005027049.1,WP_005027120.1,
        WP_111281831.1,WP_005406549.1,WP_111281978.1,WP_111282072.1,WP_111282809.1,
        WP_111282131.1,WP_111282142.1,WP_005018919.1,WP_111282175.1,WP_111282201.1,
        WP_005018794.1,WP_046205772.1,WP_005027688.1,WP_106439128.1,WP_005018365.1,
        WP_111282411.1,WP_111282415.1,WP_005024277.1,WP_005017910.1,WP_111282564.1,
        WP_111282582.1,WP_005024026.1,WP_111282679.1,WP_026056665.1
        
        
        ''')
        
        put_collapse('Deinococcus DdrO',
        '''
        Input Taxonomy:
        Deinococcus

        NCBI Protein ID:
        WP_012695046.1,WP_012694483.1,WP_049760561.1,WP_012692255.1,WP_012693173.1,
        WP_162485426.1,WP_012694690.1,WP_012693159.1,WP_012692693.1,WP_012691930.1,
        WP_012693877.1,WP_012693497.1

        ''')

        
    put_text('Hope you like it!\nThis website is maintained by shuang_s@zju.edu.cn')
    
    info = input_group("Novel cis-element discovery",[
        input('Input Taxonomy', name='tax'),
        input('Input NCBI Protein ID', name='protein_id'),
        input('Genes in one combination(0 use only comparative information,1 for gene-specific motif discovery)', name='set_length')
        
  #input('Input Representative Species', name='select_specie')
    ])
    
    #put_text(info['tax'],info['protein_id'],info['select_specie'])
    

        
    remove('scope1')
    

    #basic information
    ntime=str(datetime.now()).replace(' ','_')
    taxonomy=info['tax'].replace(' ','')
    #select_specie=info['select_specie'].replace(' ','').split('_')[1].split('.')[0]#
    proteins=info['protein_id'].replace(' ','')
    protein_list=proteins.split(',')
    set_length=int(info['set_length'])
    
    
    put_text('Input Taxonomy is '+taxonomy)
    put_text('Input NCBI Protein ID:'+proteins)
    put_text(str(set_length)+' Genes in one combination')
    
    check_result=check_before_task(taxonomy,protein_list,set_length)
    print(taxonomy)
    #if taxonomy=='Chordata':
    #    check_result='True'
    #XX
    #check_result='True'
    print(check_result)
    if check_result!='True':
        put_text('Here:'+check_result)
        raise
    put_text('Check passed!')
    put_processbar('bar')
    set_processbar('bar', 0.5 / 10)
    put_text(ntime)
    
    #processing
    if taxonomy not in os.listdir('/data/songshuang/server_find_novel_regulator/0_gbk'):
        download_gbk(taxonomy)
        extract_protein(taxonomy)
        diamond_task(taxonomy)
    set_processbar('bar', 2 / 10)
    get_selected_protein_diamond(protein_list,taxonomy,ntime)
    set_processbar('bar', 4 / 10)
    combine_meme_work(ntime,set_length)
    set_processbar('bar', 9 / 10)
    motif_for_R(ntime)
    set_processbar('bar', 10 / 10)
    put_text(str(datetime.now()).replace(' ','_'))
    
    #download the result
    os.chdir('/data/songshuang/server_find_novel_regulator/99_User_project/%s/3_meme_result/'%ntime)
    os.system('mv -r ../2_promoter ./promoter'/)
    try:
        os.system('tar -czvf %s.tar.gz *'%(taxonomy))
        img = open('/data/songshuang/server_find_novel_regulator/99_User_project/%s/3_meme_result/meme/novel_motif.png'%ntime, 'rb').read()  
        put_image(img)
    except:
        put_text('There is no significant result.')
    try:
        #pal
        put_text('Palindrome sequence')
        img1 = open('/data/songshuang/server_find_novel_regulator/99_User_project/%s/3_meme_result/pal/novel_motif.png'%ntime, 'rb').read()  
        put_image(img1)
    except:
        put_text('There is no significant result.')

    content = open('./%s.tar.gz'%taxonomy, 'rb').read()
    put_file('%s.tar.gz'%taxonomy, content, 'download me')





In [ ]:
if __name__ == '__main__':
    start_server(web_work,port=8848)